In [ ]:
import numpy as np
import pandas as pd
import re
from collections import Counter
import matplotlib.pylab as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import os
import formula_reader

In [ ]:
def cleaning_the_dataset(dataset_material):
    for ii in dataset_material.keys():
        if (ii.find('Unnamed') != -1):
            del dataset_material[ii]
        elif list(set(pd.isnull(dataset_material[ii].values)))[0]==True:
            del dataset_material[ii]
        else:
            pass

In [ ]:
dataset_material = pd.read_excel('Data_Ghiringhelli.xlsx', 'Material Data')
dataset_atomic = pd.read_excel('Data_Ghiringhelli.xlsx',   'Atomic Data')
dataset_atomic.columns = ['Z', 'A', 'IP', 'EA', 'HOMO', 'LUMO', 'rs', 'rp', 'rd', '1D']
dataset_test = pd.read_csv('dataset_test.csv')
dataset_test_new = pd.read_csv('dataset_test_new.csv')

In [ ]:
cleaning_the_dataset(dataset_atomic)
cleaning_the_dataset(dataset_material)

In [ ]:
column_name = formula_reader.columns_name(dataset_atomic_columns=dataset_atomic.keys()[2:-1], 
                           num_of_element=['A','B'])

In [ ]:
equation_list = ['(rp_B**3 - exp(rs_B))/((rp_A**2))', 
                 '(sqrt(fabs(EA_A))+sqrt(fabs(IP_B)))/((rp_A**2))']

In [ ]:
df_form = pd.read_excel('Final wrap up.xlsx', sheet_name='ZincBlend 1D')

all_num,all_equations,all_avg_mse = [],[],[]
for str_lst in df_form['OAD_gen_1 RMSE']:
    try :
        lindex = str_lst.index('(')
        rindex = str_lst.rindex(')')
        equation=str_lst[lindex:rindex+1]
        all_equations.append(equation)
    except:
        pass

In [ ]:
new_equation_list, single_mse=[],[]
coeff_list, new_single_mse   =[],[]
max_coeff_list, max_single_mse=[],[]
min_slops, min_intercepts = [],[]
regression = 'On'
rounding = 12
target_property = 'classification_accuracy' # 'RMSE' or 'classification_accuracy'

actute_species = ['RS' if i < 0 else 'ZB' for i in dataset_test.DE]
def target_func(target_prop, pred, actu):
    if target_prop == 'RMSE':
        return np.round(np.sqrt(mean_squared_error(pred, 
                                 actu)),rounding)
    
    elif target_prop == 'classification_accuracy':
        pred_species = ['RS' if i < 0 else 'ZB' for i in pred]
        quant_res = ([i for i, j in zip(actute_species, pred_species) if i == j])
        return (len(quant_res)*100/82)
    
    else:
        raise Exception('Please select target_prop: "RMSE" or "classification_accuracy"')
        
for equation in all_equations[:10]:#+all_equations[4:7]+all_equations[8:9]:
    slops, intercepts = [],[]
    equation = equation.replace('HOMOKS','HOMO')
    equation = equation.replace('LUMOKS','LUMO')
    eq_obj = formula_reader.equation_reader(equation_string=equation, 
                                            dataset=dataset_test)
    feature_array = np.array(eq_obj.run_equation_reader())
    feature_array = np.reshape(feature_array,(-1,1))
    a,b,c = eq_obj.get_individual_element()
    formula = eq_obj.get_formula()
    new_equation_list.append(formula)
    actu = dataset_test['DE'].values

    regressor = LinearRegression()
    regressor.fit(feature_array, actu)
    
    pred = regressor.predict(np.array(feature_array))
    single_mse.append(target_func(target_prop= target_property, 
                                                    pred=pred, actu=actu))
    print(single_mse)
    coeff = regressor.coef_
    intercept = regressor.intercept_
    
    temp_mse, value_lst = [],[]
    for ii in np.arange(-1.1,1.1,0.01):
        for jj in np.arange(-1.1,1.1,0.01):
                if np.round(ii,4)==0. or np.round(jj,4)==0.:
                    pass
                else:
                    x = (np.round(ii,4)*a + np.round(jj,4)*b)/(c)
                    if regression == 'On':
                        regressor = LinearRegression()
                        regressor.fit(np.array(x).reshape(-1,1), dataset_test.DE.values)
                        pred = regressor.predict(np.array(x).reshape(-1,1))
                        temp_mse.append(target_func(target_prop= target_property, 
                                                    pred=pred, actu=actu))
                        value_lst.append([np.round(ii,4), np.round(jj,4)])
                        slops.append(regressor.coef_)
                        intercepts.append(regressor.intercept_)
                    else:
                        pred = (coeff*x)+intercept
                        temp_mse.append(target_func(target_prop=target_property, 
                                                    pred=pred, actu=actu))
                        value_lst.append([np.round(ii,4), np.round(jj,4)])
    
    if target_property == 'RMSE':
        args = np.argmin(temp_mse)
        error = min(temp_mse)
    
    elif target_property == 'classification_accuracy':
        args = np.argmax(temp_mse)
        error = max(temp_mse)
    
    print(value_lst[args], error)
    diff=  [0.1,0.01,0.001,0.0001]
    interval = [0.01,0.001,0.0001,0.00001]
    for ll in range(len(diff)):
        range_lst = value_lst[args]
        for ii in (np.arange(range_lst[0]-diff[ll],
                             range_lst[0]+diff[ll],interval[ll])):
            for jj in (np.arange(range_lst[1]-diff[ll], 
                                 range_lst[1]+diff[ll],interval[ll])):
                    if np.round(ii,4)==0. or np.round(jj,4)==0.:
                        pass
                    else:
                        x = (np.round(ii,4)*a + np.round(jj,4)*b)/(c)
                        if regression == 'On':
                            regressor = LinearRegression()
                            regressor.fit(np.array(x).reshape(-1,1), dataset_test.DE.values)
                            pred = regressor.predict(np.array(x).reshape(-1,1))
                            temp_mse.append(target_func(target_prop=target_property, 
                                                        pred=pred, actu=actu))
                            value_lst.append([np.round(ii,4), np.round(jj,4)])
                            slops.append(regressor.coef_)
                            intercepts.append(regressor.intercept_)
                        else:                            
                            pred = (coeff*x)+intercept
                            temp_mse.append(target_func(target_prop=target_property, 
                                                        pred=pred, actu=actu))
                            value_lst.append([np.round(ii,4), np.round(jj,4)])
        print(value_lst[args], error)    
    
    print('------------------------------------',len(temp_mse))
    new_single_mse.append(error)
    coeff_list.append(value_lst[args])
    max_single_mse.append(error)
    max_coeff_list.append(value_lst[args])
    min_slops.append(slops[args])
    min_intercepts.append(intercepts[args])

In [ ]:
df = pd.DataFrame()
df['Equations'] = new_equation_list
df['old_mse_for_full_dataset']=single_mse
df['rmse_after_optimization_for_full_dataset']=new_single_mse
df['individual_coeff'] = coeff_list
df['slops'] = min_slops
df['intecepts'] = min_intercepts
#df['classification accuracy'] = temp